# 여러 문서를 조합해서 답변해야하는 복합 질문 생성

In [ ]:
import random
import ast
import pandas as pd
from openai import OpenAI
from google.colab import userdata
import os
import re
from tqdm import tqdm

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
# klue_mrc_context_question_negative_samples.csv
!gdown 1Jy5FX41lSF9VH-LUeCPTgDAn9flXQqHc

Downloading...
From (original): https://drive.google.com/uc?id=1Jy5FX41lSF9VH-LUeCPTgDAn9flXQqHc
From (redirected): https://drive.google.com/uc?id=1Jy5FX41lSF9VH-LUeCPTgDAn9flXQqHc&confirm=t&uuid=e68a8c84-e1ba-4527-88ad-ff72891cd10b
To: /content/klue_mrc_context_question_negative_samples.csv
100% 161M/161M [00:01<00:00, 89.5MB/s]


In [ ]:
# CSV 파일을 DataFrame으로 읽기
df = pd.read_csv('klue_mrc_context_question_negative_samples.csv')

# 3000~3200번째 행 추출
df_200 = df.iloc[3000:3200].reset_index(drop=True)
df_200

,context,question,negative_samples
0,박근혜 대통령이 17일 개각을 통해 내정한 4명의 장관 후보자 중 2명은 정치인이다...,개각 전 내각 구성원 중 정치인은 총 몇 명이었나?,['박근혜 대통령이 21일 총선에 출마하는 현직 장관 다섯 명과 임기가 만료된 국민...
1,전국에서 1인당 연간 진료비 지출이 가장 많은 곳은 전북 부안군인 것으로 나타났다....,지난해 연간 1인당 진료비 지출이 세 번째로 높은 지역의 액수는?,['지난해 건강보험 총진료비가 처음으로 50조원(50조9552억원)을 넘어섰다. 이...
2,“변수가 많아 수급 상황은 예상하기 어렵습니다.”(박성욱 SK하이닉스 사장) “내년...,SK하이닉스 제품 중 가격이 하락한 제품은?,['지난 4일 발생한 SK하이닉스의 중국 우시 공장 화재로 이 회사 주력 제품인 D...
3,대부업체에 기반을 둔 OK저축은행과 웰컴저축은행의 성장세가 가파르다. 여신과 수신은...,예신저축은행을 인수한 은행의 총자산은?,['금융당국이 저축은행 인수를 앞둔 대부업체인 에이앤피파이낸셜그룹(러시앤캐시)과 웰...
4,간토 지진( ) 또는 사가미 해곡 거대지진( |사가미토라후쿄다이지신}})은 일본의 ...,간토 대지진 외에 사가미 해곡 지진에 해당되는 지진은?,['난카이 해곡의 지진은 약 90년-150년(중세 이전에는 200년 이상)의 간격으...
...,...,...,...
195,가라지는 체고가 낮고 몸은 방추형이며 옆으로 납작하며 양턱에 미세한 이빨이 나있다....,가라지가 주로 서식하는 곳은?,"[""지중해 연안은 기온적으로는 조엽수림대와 거의 비슷하지만, 연강수량이 1,000m..."
196,시카고가 미국의 주요 대도시 가운데 처음으로 에너지 음료 규제를 추진하고 나섰다. ...,사망 사건으로 논란이 된 에너지 드링크는?,"['나발니 음독 사건\n\n2020년 8월 20일, 시베리아 톰스크에서 모스크바로 ..."
197,모하메드 레자 팔라비는 제2차 세계 대전 와중 영국과 소련이 이란을 침공했을 때 부...,1979년에 정치범들이 일으킨 변혁의 이름은 무엇인가?,"['1979년, 격해지는 반미 시위 가운데 이란 시위대가 주 이란 미국 대사관 벽을..."
198,중국 정부의 시중 자금 회수 등 ‘은행 길들이기’로 촉발된 유동성 위기가 실물경제에...,최근 중국 기업의 1년짜리 회사채에 대한 할인율은 몇 퍼센트인가?,['중국이 두 달여 만에 기준금리 인하 카드를 또 꺼내들었다. 올해 중국의 경제성장...


In [ ]:
# 각 row에 대해 docs 리스트를 만들고 DataFrame에 저장
def make_docs(row):
    negatives = row['negative_samples']
    # negative_samples가 str이면 리스트로 변환
    if isinstance(negatives, str):
        negatives = ast.literal_eval(negatives)
    docs = [row['context']] + negatives
    random.shuffle(docs)
    return docs

df_200['docs'] = df_200.apply(make_docs, axis=1)
df_200[['context', 'question', 'negative_samples', 'docs']].head()

,context,question,negative_samples,docs
0,박근혜 대통령이 17일 개각을 통해 내정한 4명의 장관 후보자 중 2명은 정치인이다...,개각 전 내각 구성원 중 정치인은 총 몇 명이었나?,['박근혜 대통령이 21일 총선에 출마하는 현직 장관 다섯 명과 임기가 만료된 국민...,[아베 신조(安倍晋三) 일본 총리(사진)가 7일 19명의 각료 가운데 문부과학상 등...
1,전국에서 1인당 연간 진료비 지출이 가장 많은 곳은 전북 부안군인 것으로 나타났다....,지난해 연간 1인당 진료비 지출이 세 번째로 높은 지역의 액수는?,['지난해 건강보험 총진료비가 처음으로 50조원(50조9552억원)을 넘어섰다. 이...,[전국에서 1인당 연간 진료비 지출이 가장 많은 곳은 전북 부안군인 것으로 나타났다...
2,“변수가 많아 수급 상황은 예상하기 어렵습니다.”(박성욱 SK하이닉스 사장) “내년...,SK하이닉스 제품 중 가격이 하락한 제품은?,['지난 4일 발생한 SK하이닉스의 중국 우시 공장 화재로 이 회사 주력 제품인 D...,[“변수가 많아 수급 상황은 예상하기 어렵습니다.”(박성욱 SK하이닉스 사장) “내...
3,대부업체에 기반을 둔 OK저축은행과 웰컴저축은행의 성장세가 가파르다. 여신과 수신은...,예신저축은행을 인수한 은행의 총자산은?,['금융당국이 저축은행 인수를 앞둔 대부업체인 에이앤피파이낸셜그룹(러시앤캐시)과 웰...,[금융당국이 저축은행 인수를 앞둔 대부업체인 에이앤피파이낸셜그룹(러시앤캐시)과 웰컴...
4,간토 지진( ) 또는 사가미 해곡 거대지진( |사가미토라후쿄다이지신}})은 일본의 ...,간토 대지진 외에 사가미 해곡 지진에 해당되는 지진은?,['난카이 해곡의 지진은 약 90년-150년(중세 이전에는 200년 이상)의 간격으...,[같은 날 기나이와 도카이도 지역에서 관측한 강진 및 쓰나미 기록이 휴가나다 지역에...


In [ ]:
system_prompt = """당신은 여러 문서를 조합해서 답변할 수밖에 없는 복합 질문을 만드는 전문가입니다.
아래 지침을 반드시 따라주세요:
- 반드시 2개 이상의 문서를 함께 인용해야만 답할 수 있는 새로운 질문을 만들어주세요.
- 질문이 왜 둘 이상의 문서(doc1, doc2, ...))가 필요한지 그 근거(Reason)를 작성해주세요.
- 결론(Conclusion)에서는 Reason에서 근거가 제대로 제시되었는지 확인하고, 맞으면 : True, 부족하면 : False를 출력하세요.
- 반드시 존댓말을 사용하세요.
"""

for idx, row in df_200.iterrows():
    # 문서 번호 매기기
    doc_items = [f"doc{i+1}: {doc}" for i, doc in enumerate(row['docs'])]
    docs_str = '\n'.join(doc_items)
    user_prompt = f"""아래의 5개 문서(doc1~doc5)를 모두 참고하세요.

docs:
{docs_str}

아래 지침을 따르세요.
1. 반드시 두 개 이상의 문서를 함께 인용하지 않으면 답할 수 없는 복합적인 Question(질문)을 만들어주세요.
2. Reason(근거)에서는 왜 두 문서 이상의 정보(doc1, doc2, ...)가 함께 필요하다고 생각하는지 설명해 주세요.
3. Conclusion(결론)에서는 Reason의 논리가 충분한지 스스로 검토 내용을 기술합니다.
4. Answer(답변)에는 반드시 True 또는 False 만으로 답해주세요.
5. 반드시 아래의 형식을 따르십시오.

[형식]
Question: ...
Reason: ...
Conclusion: ...
Answer: True/False
"""
    break

print(user_prompt)


아래의 5개 문서(doc1~doc5)를 모두 참고하세요.

docs:
doc1: 아베 신조(安倍晋三) 일본 총리(사진)가 7일 19명의 각료 가운데 문부과학상 등 10명을 교체하는 개각을 단행했다. 재무 외무 등 핵심 각료 9명은 자리를 지켰다.아베 총리는 이날 각의를 거쳐 새 내각의 각료 19명을 확정했다. 신임 문부과학상에는 하세 히로시 중의원, 신설된 ‘1억 총활약사회담당상’에는 가토 가쓰노부 관방 부(副)장관이 발탁됐다. 프로레슬러 출신인 하세 신임 문부과학상은 군위안부 강제연행을 부정하고 극우 성향의 역사 교과서를 지지해 온 인물이다.‘1억 총활약사회’는 아베 총리가 지난달 자유민주당 총재 연임을 확정한 뒤 내놓은 구호로, 일본인 전체가 각 분야에서 활약할 수 있는 사회를 이루고 50년 뒤에도 인구 1억명을 유지한다는 목표를 담고 있다.국토교통상에는 이시이 게이치 중의원, 농림수산상에 모리야마 히로시 중의원, 경제산업상에는 하야시 모토오 중의원이 각각 기용됐다. 부흥상은 다카기 쓰요시 중의원, 법무상은 이와키 미쓰히데 참의원, 행정개혁담당상은 고노 다로 중의원이 새로 맡았다. 아소 다로 부총리 겸 재무상, 기시다 후미오 외무상 등 9명은 유임됐다. 도쿄=서정환  특파원
doc2: 박근혜 대통령이 17일 개각을 통해 내정한 4명의 장관 후보자 중 2명은 정치인이다. 유일호 국토교통부 장관 후보자는 재선 의원(서울 송파을)이며, 유기준 해양수산부 장관 후보자는 3선 의원(부산 서구)이다. 이로써 ‘이완구 내각’ 멤버 중 3분의 1이 정치인으로 채워졌다. 여권 관계자는 “정치인 출신이 이렇게 내각에 다수 포진하는 것은 역대 정부에서도 드문 사례”라고 말했다. 때문에 일각에선 사실상 ‘의원내각제’가 아니냐는 얘기도 나온다.○사실상 의원내각제?내각 구성원 중 총리를 포함해 장관은 모두 18명이다. 이번 개각으로 정치인은 기존 3명(최경환 부총리 겸 기획재정부 장관, 황우여 부총리 겸 교육부 장관, 김희정 여성가족부 장관)에서 6명으로 늘어났다. 총리를 포함해 

In [ ]:
client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4.1-mini-2025-04-14",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.5
)

print(response.choices[0].message.content)

Question: 아베 신조 일본 총리의 2014년 개각에서 문부과학상과 ‘1억 총활약사회담당상’이 신설 및 교체된 배경과, 박근혜 대통령이 2014년 개각에서 정치인 출신 장관을 대거 기용한 이유 및 그에 따른 내각 구성 변화는 어떻게 비교할 수 있습니까?

Reason: 이 질문은 아베 신조 일본 총리의 개각 내용과 배경을 다룬 doc1(및 doc3)과 박근혜 대통령의 개각에서 정치인 출신 장관의 대거 기용과 그 의미를 다룬 doc2(및 doc4)를 함께 참고해야만 답변할 수 있습니다. doc1과 doc3는 아베 총리의 개각에서 신설된 ‘1억 총활약사회담당상’과 문부과학상 교체 인사 및 이들의 정치적 성향과 정책 목표를 설명하고 있으나, 정치인 출신 장관 기용 배경이나 내각 구성 변화에 관한 내용은 포함되어 있지 않습니다. 반면 doc2와 doc4는 박근혜 대통령의 개각에서 정치인 출신 장관이 늘어난 이유, 국회와의 소통 강화 목적, 총선 출마와 관련한 내각 변화, 그리고 친박계 인사 발탁 배경 등을 상세히 다루고 있어 아베 총리 개각과 비교하기 위해서는 두 국가의 개각 배경과 구성을 모두 파악해야 합니다. 따라서 두 개 이상의 문서가 필요합니다.

Conclusion: Reason에서 아베 신조 총리의 개각 내용과 박근혜 대통령의 개각 배경 및 정치인 기용 이유가 각각 다른 문서에 분산되어 있어 두 문서를 모두 참고해야만 비교 분석이 가능하다는 점을 명확히 제시하고 있으므로 논리가 충분하다고 판단됩니다.

Answer: True


In [ ]:
composite_qs = []

for idx, row in tqdm(df_200.iterrows(), total=len(df_200), desc="복합질문 생성 중"):
    doc_items = [f"doc{i+1}: {doc}" for i, doc in enumerate(row['docs'])]
    docs_str = '\n'.join(doc_items)
    user_prompt = f"""아래의 5개 문서(doc1~doc5)를 모두 참고하세요.

docs:
{docs_str}

아래 지침을 따르세요.
1. 반드시 두 개 이상의 문서를 함께 인용하지 않으면 답할 수 없는 복합적인 Question(질문)을 만들어주세요.
2. Reason(근거)에서는 왜 두 문서 이상의 정보(doc1, doc2, ...)가 함께 필요하다고 생각하는지 설명해 주세요.
3. Conclusion(결론)에서는 Reason의 논리가 충분한지 스스로 검토 내용을 기술합니다.
4. Answer(답변)에는 반드시 True 또는 False 만으로 답해주세요.
5. 반드시 아래의 형식을 따르십시오.

[형식]
Question: ...
Reason: ...
Conclusion: ...
Answer: True/False
"""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.5
    )
    composite_qs.append(response.choices[0].message.content)

df_200['composite_question'] = composite_qs


복합질문 생성 중: 100%|██████████| 200/200 [16:18<00:00,  4.89s/it]


In [ ]:
print(df_200['composite_question'][10])

[형식]  
Question: 통합진보당 해산 사건과 관련하여, 노동조합의 파업과 정치적 대응이 어떻게 연결될 수 있는지 설명해 주실 수 있나요?  
Reason: 이 질문은 doc1에서 통합진보당 해산과 관련된 정치적 사건을 다루고 있으며, doc2에서는 노동조합의 파업과 그에 대한 정치적 반응을 설명하고 있습니다. 두 문서 모두 정치적 갈등과 사회적 저항을 다루고 있지만, 각각의 맥락이 다르기 때문에 두 문서를 함께 참고해야만 통합적인 이해가 가능하기 때문입니다.  
Conclusion: Reason에서 제시된 논리가 충분하며, 두 문서의 내용을 결합해야만 질문에 대한 명확한 답변을 도출할 수 있습니다.  
Answer: True


In [ ]:
def filter_answer_true(df, column='composite_question'):
    """
    composite_question 컬럼에서 Answer: False로 끝나는 행을 제거합니다.
    """
    # True/False 구분은 대소문자 상관없이 'Answer: False'로 끝나는지 체크
    return df[df[column].apply(lambda x: not re.search(r'Answer:\s*False\s*$', str(x), re.IGNORECASE))].reset_index(drop=True)


# False 행 제거
df_200_true = filter_answer_true(df_200, column='composite_question')

print(df_200_true.shape)
print(df_200_true['composite_question'].iloc[0])

(200, 5)
Question: 아베 신조 총리의 개각과 박근혜 대통령의 개각이 각각 어떤 정치적 배경과 목적을 가지고 이루어졌는지 비교하여 설명해 주실 수 있습니까?

Reason: 아베 신조 총리의 개각에 대한 정보는 doc1과 doc3에서 제공되며, 박근혜 대통령의 개각에 대한 정보는 doc2와 doc4에서 확인할 수 있습니다. 이 두 그룹의 문서들은 각각 일본과 한국의 정치적 맥락에서의 개각의 배경과 목적을 다루고 있어, 두 가지 사례를 비교하기 위해서는 두 개 이상의 문서가 필요합니다.

Conclusion: Reason에서 제시된 논리는 두 개 이상의 문서가 필요한 이유를 명확히 설명하고 있습니다. 따라서, 비교 분석을 위해 제시된 문서들이 충분히 활용될 수 있습니다. 
Answer: True


In [ ]:
def extract_question(text):
    """
    composite_question 문자열에서 Question: ... 부분만 추출
    """
    if not isinstance(text, str):
        return ""
    # 정규표현식: Question: ~ (다음 라인에서 끝남)
    match = re.search(r'Question:\s*(.+)', text)
    if match:
        # 줄바꿈(\n) 이전까지만
        return match.group(1).split('\n')[0].strip()
    else:
        return ""


df_200_true['question_only'] = df_200_true['composite_question'].apply(extract_question)

# 결과 확인
df_200_true[['composite_question', 'question_only']].head()

,composite_question,question_only
0,Question: 아베 신조 총리의 개각과 박근혜 대통령의 개각이 각각 어떤 정치적...,아베 신조 총리의 개각과 박근혜 대통령의 개각이 각각 어떤 정치적 배경과 목적을 가...
1,[형식]\nQuestion: 한국의 지역별 1인당 평균 진료비와 외국인 환자의 평균...,한국의 지역별 1인당 평균 진료비와 외국인 환자의 평균 진료비 차이를 분석하기 위해...
2,[형식] \nQuestion: SK하이닉스의 우시 공장 화재로 인한 D램 가격 상...,SK하이닉스의 우시 공장 화재로 인한 D램 가격 상승세와 삼성전자의 경쟁력에 대한 ...
3,Question: 현재 대부업체와 저축은행 간의 인수합병 및 자산 축소에 대한 금융...,현재 대부업체와 저축은행 간의 인수합병 및 자산 축소에 대한 금융당국의 요구사항은 ...
4,Question: 1498년의 난카이 지진과 1923년 간토 대지진의 발생 원인 및...,1498년의 난카이 지진과 1923년 간토 대지진의 발생 원인 및 피해 양상에 대한...


In [ ]:
df_200_true['question_only'][0]

'아베 신조 총리의 개각과 박근혜 대통령의 개각이 각각 어떤 정치적 배경과 목적을 가지고 이루어졌는지 비교하여 설명해 주실 수 있습니까?'

In [ ]:
system_prompt = """당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, 그리고 문서 내에서만 정보를 근거로 하여 답변하는 AI 비서입니다.
아래 지침을 반드시 지켜주세요:

- 답변은 반드시 docs에서 찾은 내용에 한해서만 작성해주세요. docs에 없는 내용은 추론하거나 지어내지 마세요.
- 답변에서 인용하는 부분이 있다면, 반드시 해당 문서의 번호(예: [[doc1]], [[doc2]])로 근거를 표시해 주세요.
- docs의 순서와 번호는 중요합니다. docs에서 인용하지 않은 정보는 답변에 포함하지 마세요.
- 답변의 근거가 되는 문서 번호를 생략하지 말고, 항상 인용 태그([[doc1]], [[doc2]], ...)를 포함해 주세요.
- 모든 답변은 존댓말을 사용하세요."""

user_prompts = []
system_prompts = []

for idx, row in df_200_true.iterrows():
    docs = row['docs']
    # 문서에 번호 붙이기
    doc_items = [f"doc{i+1}: {doc}" for i, doc in enumerate(docs)]
    docs_str = '\n'.join(doc_items)
    # user_prompt 생성
    user_prompt = f"""질문: {row['question_only']}

docs:
{docs_str}

위의 docs 중에서만 정보를 근거로 하여, 질문에 답변해 주세요.
답변에서 인용한 문서의 내용에는 반드시 [[doc1]], [[doc2]], ... 형식으로 인용 표시를 해주세요.
추론이나 지어내는 답변은 삼가주시고, docs에 명시적으로 나타난 내용만 인용해 주세요."""
    # 리스트에 추가
    user_prompts.append(user_prompt)
    system_prompts.append(system_prompt)

# 예시 출력
print("system_prompt:")
print(system_prompts[100])
print("\nuser_prompt:")
print(user_prompts[100])

system_prompt:
당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, 그리고 문서 내에서만 정보를 근거로 하여 답변하는 AI 비서입니다.
아래 지침을 반드시 지켜주세요:

- 답변은 반드시 docs에서 찾은 내용에 한해서만 작성해주세요. docs에 없는 내용은 추론하거나 지어내지 마세요.
- 답변에서 인용하는 부분이 있다면, 반드시 해당 문서의 번호(예: [[doc1]], [[doc2]])로 근거를 표시해 주세요.
- docs의 순서와 번호는 중요합니다. docs에서 인용하지 않은 정보는 답변에 포함하지 마세요.
- 답변의 근거가 되는 문서 번호를 생략하지 말고, 항상 인용 태그([[doc1]], [[doc2]], ...)를 포함해 주세요.
- 모든 답변은 존댓말을 사용하세요.

user_prompt:
질문: 삼성전자가 개발한 스마트폰 카메라 모듈의 기술적 발전과 그에 따른 소비자 반응은 어떤 관계가 있으며, 이를 통해 어떤 제품들이 시장에 출시되었는지 설명해 주실 수 있나요?

docs:
doc1: 삼성전자는 스마트폰 카메라 모듈의 두께를 지금보다 20%가량 줄일 수 있는 모바일 이미지 센서(사진)를 업계 최초로 양산하는 데 성공했다고 29일 발표했다.이 이미지 센서는 화소 크기가 1.0㎛(마이크로미터·100만분의 1m)로 기존 제품(1.12㎛)보다 얇다. 화소 크기를 줄이면 흡수하는 빛의 양이 감소해 화질이 떨어질 수 있지만 삼성전자는 화소를 격리해 간섭 현상을 최소화하는 기술로 빛 손실을 줄여 1.0㎛ 화소로 기존 1.12㎛ 화소와 같은 수준의 화질을 구현했다.이번에 양산한 이미지 센서를 사용하면 화질 감소 없이 카메라 모듈 두께를 5㎜ 이하로 줄일 수 있다. 카메라 모듈 두께가 종전보다 20%가량 얇아진다. 이 카메라 모듈을 초슬림 스마트폰에 장착하면 밖으로 튀어나오는 부분이 거의 없다.신제품은 삼성전자가 이달 초부터 판매한 중저가폰 갤럭시A8에 사용됐다. 삼성전자가 다음달 13일 공개할 하반기 전략 스마트폰 갤럭

In [ ]:
df_200_true['user_prompt'] = user_prompts
df_200_true['system_prompt'] = system_prompts
df_200_true[['question_only', 'docs', 'system_prompt', 'user_prompt']].head()

,question_only,docs,system_prompt,user_prompt
0,아베 신조 총리의 개각과 박근혜 대통령의 개각이 각각 어떤 정치적 배경과 목적을 가...,[아베 신조(安倍晋三) 일본 총리(사진)가 7일 19명의 각료 가운데 문부과학상 등...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",질문: 아베 신조 총리의 개각과 박근혜 대통령의 개각이 각각 어떤 정치적 배경과 목...
1,한국의 지역별 1인당 평균 진료비와 외국인 환자의 평균 진료비 차이를 분석하기 위해...,[전국에서 1인당 연간 진료비 지출이 가장 많은 곳은 전북 부안군인 것으로 나타났다...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",질문: 한국의 지역별 1인당 평균 진료비와 외국인 환자의 평균 진료비 차이를 분석하...
2,SK하이닉스의 우시 공장 화재로 인한 D램 가격 상승세와 삼성전자의 경쟁력에 대한 ...,[“변수가 많아 수급 상황은 예상하기 어렵습니다.”(박성욱 SK하이닉스 사장) “내...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",질문: SK하이닉스의 우시 공장 화재로 인한 D램 가격 상승세와 삼성전자의 경쟁력에...
3,현재 대부업체와 저축은행 간의 인수합병 및 자산 축소에 대한 금융당국의 요구사항은 ...,[금융당국이 저축은행 인수를 앞둔 대부업체인 에이앤피파이낸셜그룹(러시앤캐시)과 웰컴...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",질문: 현재 대부업체와 저축은행 간의 인수합병 및 자산 축소에 대한 금융당국의 요구...
4,1498년의 난카이 지진과 1923년 간토 대지진의 발생 원인 및 피해 양상에 대한...,[같은 날 기나이와 도카이도 지역에서 관측한 강진 및 쓰나미 기록이 휴가나다 지역에...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",질문: 1498년의 난카이 지진과 1923년 간토 대지진의 발생 원인 및 피해 양상...


In [ ]:
client = OpenAI()

# 응답 저장 리스트
responses = []

for i, user_prompt in enumerate(tqdm(user_prompts, desc="응답 생성 중", ncols=80)):
    response = client.chat.completions.create(
        model="gpt-4.1-mini-2025-04-14",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    answer = response.choices[0].message.content
    responses.append(answer)

응답 생성 중: 100%|███████████████████████████| 200/200 [37:55<00:00, 11.38s/it]


In [ ]:
def extract_doc_citations(text):
    """
    [[doc1]], [[doc2]], ... 형식의 인용 문서번호만 리스트로 반환
    """
    # 정규표현식: 대소문자 구분 없이 [[doc숫자]]
    citations = re.findall(r'\[\[(doc\d+)\]\]', text, flags=re.IGNORECASE)
    return sorted(set(citations))  # 중복 제거

In [ ]:
# 1. answer 컬럼 추가
df_200_true['answer'] = responses

# 2. doc_citations 컬럼 추가
df_200_true['doc_citations'] = df_200_true['answer'].apply(extract_doc_citations)

# 3. 결과 일부 확인
df_200_true[['question_only', 'answer', 'docs', 'doc_citations']].head()

,question_only,answer,docs,doc_citations
0,아베 신조 총리의 개각과 박근혜 대통령의 개각이 각각 어떤 정치적 배경과 목적을 가...,아베 신조 총리의 개각과 박근혜 대통령의 개각은 각각 다른 정치적 배경과 목적을 가...,[아베 신조(安倍晋三) 일본 총리(사진)가 7일 19명의 각료 가운데 문부과학상 등...,"[doc1, doc2, doc3, doc4]"
1,한국의 지역별 1인당 평균 진료비와 외국인 환자의 평균 진료비 차이를 분석하기 위해...,한국의 지역별 1인당 평균 진료비와 외국인 환자의 평균 진료비 차이를 분석하기 위해...,[전국에서 1인당 연간 진료비 지출이 가장 많은 곳은 전북 부안군인 것으로 나타났다...,"[doc1, doc2, doc3, doc4]"
2,SK하이닉스의 우시 공장 화재로 인한 D램 가격 상승세와 삼성전자의 경쟁력에 대한 ...,SK하이닉스의 중국 우시 공장 화재로 인해 PC용 D램 가격이 지속적으로 상승하는 ...,[“변수가 많아 수급 상황은 예상하기 어렵습니다.”(박성욱 SK하이닉스 사장) “내...,"[doc1, doc2, doc4]"
3,현재 대부업체와 저축은행 간의 인수합병 및 자산 축소에 대한 금융당국의 요구사항은 ...,금융당국은 대부업체인 에이앤피파이낸셜그룹(러시앤캐시)과 웰컴크레디라인대부(웰컴론)에...,[금융당국이 저축은행 인수를 앞둔 대부업체인 에이앤피파이낸셜그룹(러시앤캐시)과 웰컴...,"[doc1, doc2]"
4,1498년의 난카이 지진과 1923년 간토 대지진의 발생 원인 및 피해 양상에 대한...,1498년의 난카이 지진과 1923년 간토 대지진의 발생 원인 및 피해 양상에 대한...,[같은 날 기나이와 도카이도 지역에서 관측한 강진 및 쓰나미 기록이 휴가나다 지역에...,[]


In [ ]:
# 인용 문서 정보가 1개 이하인 행 제거 (즉, doc_citations가 1개 이하인 행 제거)
df_with_citation = df_200_true[df_200_true['doc_citations'].apply(lambda x: len(x) > 1)].reset_index(drop=True)

print(f"인용 정보가 2개 이상인 행만 남은 데이터 shape: {df_with_citation.shape}")

인용 정보가 2개 이상인 행만 남은 데이터 shape: (194, 10)


In [ ]:
df_with_citation[['question_only', 'docs', 'user_prompt', 'system_prompt', 'answer', 'doc_citations']]

,question_only,docs,user_prompt,system_prompt,answer,doc_citations
0,아베 신조 총리의 개각과 박근혜 대통령의 개각이 각각 어떤 정치적 배경과 목적을 가...,[아베 신조(安倍晋三) 일본 총리(사진)가 7일 19명의 각료 가운데 문부과학상 등...,질문: 아베 신조 총리의 개각과 박근혜 대통령의 개각이 각각 어떤 정치적 배경과 목...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",아베 신조 총리의 개각과 박근혜 대통령의 개각은 각각 다른 정치적 배경과 목적을 가...,"[doc1, doc2, doc3, doc4]"
1,한국의 지역별 1인당 평균 진료비와 외국인 환자의 평균 진료비 차이를 분석하기 위해...,[전국에서 1인당 연간 진료비 지출이 가장 많은 곳은 전북 부안군인 것으로 나타났다...,질문: 한국의 지역별 1인당 평균 진료비와 외국인 환자의 평균 진료비 차이를 분석하...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",한국의 지역별 1인당 평균 진료비와 외국인 환자의 평균 진료비 차이를 분석하기 위해...,"[doc1, doc2, doc3, doc4]"
2,SK하이닉스의 우시 공장 화재로 인한 D램 가격 상승세와 삼성전자의 경쟁력에 대한 ...,[“변수가 많아 수급 상황은 예상하기 어렵습니다.”(박성욱 SK하이닉스 사장) “내...,질문: SK하이닉스의 우시 공장 화재로 인한 D램 가격 상승세와 삼성전자의 경쟁력에...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",SK하이닉스의 중국 우시 공장 화재로 인해 PC용 D램 가격이 지속적으로 상승하는 ...,"[doc1, doc2, doc4]"
3,현재 대부업체와 저축은행 간의 인수합병 및 자산 축소에 대한 금융당국의 요구사항은 ...,[금융당국이 저축은행 인수를 앞둔 대부업체인 에이앤피파이낸셜그룹(러시앤캐시)과 웰컴...,질문: 현재 대부업체와 저축은행 간의 인수합병 및 자산 축소에 대한 금융당국의 요구...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",금융당국은 대부업체인 에이앤피파이낸셜그룹(러시앤캐시)과 웰컴크레디라인대부(웰컴론)에...,"[doc1, doc2]"
4,"줄기세포 치료제가 뇌전증과 조현증 치료에 미치는 잠재적 효과는 무엇인지, 그리고 이...",[국내 줄기세포 치료제 시장이 기지개를 켜고 있다. 월별 줄기세포 시술건수가 꾸준히...,"질문: 줄기세포 치료제가 뇌전증과 조현증 치료에 미치는 잠재적 효과는 무엇인지, 그...","당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",주어진 문서들에서는 줄기세포 치료제가 뇌전증과 조현증 치료에 미치는 잠재적 효과에 ...,"[doc1, doc2, doc3]"
...,...,...,...,...,...,...
189,가라지와 백로 및 왜가리의 생태적 특성과 서식지 변화가 생물 다양성에 미치는 영향은...,[가라지는 체고가 낮고 몸은 방추형이며 옆으로 납작하며 양턱에 미세한 이빨이 나있다...,질문: 가라지와 백로 및 왜가리의 생태적 특성과 서식지 변화가 생물 다양성에 미치는...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...","가라지, 백로, 왜가리의 생태적 특성과 서식지 변화가 생물 다양성에 미치는 영향에 ...","[doc1, doc2]"
190,"다양한 사건에서의 의도적인 신원 감추기와 관련된 사례를 비교 분석하고, 이러한 행동...","[1970년 11월 29일, 베르겐 근교에 위치한 울리켄(Ulriken) 산 북쪽의...","질문: 다양한 사건에서의 의도적인 신원 감추기와 관련된 사례를 비교 분석하고, 이러...","당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...","의도적인 신원 감추기와 관련된 사례를 비교 분석하고, 이러한 행동이 개인의 안전 및...","[doc1, doc4]"
191,1979년 이란 혁명과 1960년대 미국의 반전 운동이 각각의 사회적 맥락에서 어떻...,"[1979년, 격해지는 반미 시위 가운데 이란 시위대가 주 이란 미국 대사관 벽을 ...",질문: 1979년 이란 혁명과 1960년대 미국의 반전 운동이 각각의 사회적 맥락에...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",1979년 이란 혁명은 모하마드 레자 팔라비 치세의 정치적 불안과 억압에서 비롯되었...,"[doc1, doc2, doc3]"
192,중국의 유동성 위기가 기업 자금 조달에 미치는 영향과 이에 대한 중국 정부의 금리 ...,[중국 정부의 시중 자금 회수 등 ‘은행 길들이기’로 촉발된 유동성 위기가 실물경제...,질문: 중국의 유동성 위기가 기업 자금 조달에 미치는 영향과 이에 대한 중국 정부의...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",중국의 유동성 위기는 기업 자금 조달에 심각한 영향을 미치고 있습니다. 은행들이 유...,"[doc1, doc4]"


In [ ]:
# 저장할 컬럼 리스트
save_cols = ['question_only', 'docs', 'user_prompt', 'system_prompt', 'answer', 'doc_citations']

# 파일로 저장 (utf-8, 인덱스 없이)
df_with_citation.to_csv(
    'klue_mrc_prompt_multidocs_answer_citations.csv',
    columns=save_cols,
    index=False,
    encoding='utf-8'
)

In [ ]:
from google.colab import files
files.download('klue_mrc_prompt_multidocs_answer_citations.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_with_citation['question_only'][0]

'아베 신조 총리의 개각과 박근혜 대통령의 개각이 각각 어떤 정치적 배경과 목적을 가지고 이루어졌는지 비교하여 설명해 주실 수 있습니까?'

In [ ]:
df_with_citation['doc_citations'][0]

['doc1', 'doc2', 'doc3', 'doc4']

In [ ]:
print(df_with_citation['docs'][0][0])

아베 신조(安倍晋三) 일본 총리(사진)가 7일 19명의 각료 가운데 문부과학상 등 10명을 교체하는 개각을 단행했다. 재무 외무 등 핵심 각료 9명은 자리를 지켰다.아베 총리는 이날 각의를 거쳐 새 내각의 각료 19명을 확정했다. 신임 문부과학상에는 하세 히로시 중의원, 신설된 ‘1억 총활약사회담당상’에는 가토 가쓰노부 관방 부(副)장관이 발탁됐다. 프로레슬러 출신인 하세 신임 문부과학상은 군위안부 강제연행을 부정하고 극우 성향의 역사 교과서를 지지해 온 인물이다.‘1억 총활약사회’는 아베 총리가 지난달 자유민주당 총재 연임을 확정한 뒤 내놓은 구호로, 일본인 전체가 각 분야에서 활약할 수 있는 사회를 이루고 50년 뒤에도 인구 1억명을 유지한다는 목표를 담고 있다.국토교통상에는 이시이 게이치 중의원, 농림수산상에 모리야마 히로시 중의원, 경제산업상에는 하야시 모토오 중의원이 각각 기용됐다. 부흥상은 다카기 쓰요시 중의원, 법무상은 이와키 미쓰히데 참의원, 행정개혁담당상은 고노 다로 중의원이 새로 맡았다. 아소 다로 부총리 겸 재무상, 기시다 후미오 외무상 등 9명은 유임됐다. 도쿄=서정환  특파원


In [ ]:
print(df_with_citation['docs'][0][1])

박근혜 대통령이 17일 개각을 통해 내정한 4명의 장관 후보자 중 2명은 정치인이다. 유일호 국토교통부 장관 후보자는 재선 의원(서울 송파을)이며, 유기준 해양수산부 장관 후보자는 3선 의원(부산 서구)이다. 이로써 ‘이완구 내각’ 멤버 중 3분의 1이 정치인으로 채워졌다. 여권 관계자는 “정치인 출신이 이렇게 내각에 다수 포진하는 것은 역대 정부에서도 드문 사례”라고 말했다. 때문에 일각에선 사실상 ‘의원내각제’가 아니냐는 얘기도 나온다.○사실상 의원내각제?내각 구성원 중 총리를 포함해 장관은 모두 18명이다. 이번 개각으로 정치인은 기존 3명(최경환 부총리 겸 기획재정부 장관, 황우여 부총리 겸 교육부 장관, 김희정 여성가족부 장관)에서 6명으로 늘어났다. 총리를 포함해 내각의 3분의 1이 정치인으로 기용된 것이다.박 대통령이 정치인을 대거 기용한 데는 몇 가지 배경이 있다는 게 여당과 청와대 안팎의 분석이다. 우선 집권 3년차를 맞아 국정 성과를 내야 한다는 판단에 따른 것이라고 여권 관계자는 말했다. 그는 “집권 2년 동안 국정 아젠다를 많이 던졌지만 아직까지 이렇다 할 성과를 거둔 게 없는 만큼 3년차에는 무엇보다 경제혁신 3개년 계획은 물론 구조개혁 과제에 집중해 성과를 내야 하는 상황”이라며 “이런 측면에서 추진력이 강한 정치인 장관이 제격이라고 판단한 것”이라고 말했다. 그는 “경제활성화 성과를 내려면 무엇보다 국회에서 관련 법안을 통과시키는 게 관건”이라며 “이런 점에서도 정치인 장관이 유리할 수 있다”고 했다.○청문회 통과 등 다목적용 인사정치인을 발탁한 것은 인적 쇄신 의지가 부족하다는 여권 내 반발을 무마하기 위해서라는 분석도 있다. 새누리당은 지난해 말 이른바 ‘정윤회 문건’ 파문 이후 청와대 참모진은 물론 내각에 대한 대폭적인 인적 쇄신을 요구해왔다. 전날 신임 총리 국회 인준이 처리된 뒤 유승민 원내대표는 개각과 관련, “국민을 실망시켜선 안 된다”며 “과감한 인적 쇄신이 필요하다”고 주장했다.정치권 관계자는 “박 대통령으로선 당의 

In [ ]:
print(df_with_citation['docs'][0][2])

아베 신조(安倍晋三) 일본 총리(사진)가 7일 19명의 각료 가운데 문부과학상 등 10명을 교체하는 개각을 단행했다. 재무 외무 등 핵심 각료 9명은 자리를 지켰다.아베 총리는 이날 각의를 거쳐 새 내각의 각료 19명을 확정했다. 신임 문부과학상에는 하세 히로시 중의원, 신설된 ‘1억 총활약사회담당상’에는 가토 가쓰노부 관방 부(副)장관이 발탁됐다. 프로레슬러 출신인 하세 신임 문부과학상은 군위안부 강제연행을 부정하고 극우 성향의 역사 교과서를 지지해 온 인물이다.‘1억 총활약사회’는 아베 총리가 지난달 자유민주당 총재 연임을 확정한 뒤 내놓은 구호로, 일본인 전체가 각 분야에서 활약할 수 있는 사회를 이루고 50년 뒤에도 인구 1억명을 유지한다는 목표를 담고 있다.국토교통상에는 이시이 게이치 중의원, 농림수산상에 모리야마 히로시 중의원, 경제산업상에는 하야시 모토오 중의원이 각각 기용됐다. 부흥상은 다카기 쓰요시 중의원, 법무상은 이와키 미쓰히데 참의원, 행정개혁담당상은 고노 다로 중의원이 새로 맡았다. 아소 다로 부총리 겸 재무상, 기시다 후미오 외무상 등 9명은 유임됐다. 도쿄=서정환 특파원


In [ ]:
print(df_with_citation['docs'][0][3])

박근혜 대통령이 21일 총선에 출마하는 현직 장관 다섯 명과 임기가 만료된 국민권익위원장을 교체하는 중폭 개각을 단행했다. 이번 개각의 핵심은 친박(친박근혜)계인 유일호 새누리당 의원을 부총리 겸 기획재정부 장관에 발탁했다는 것이다. 유 부총리 후보자는 국토교통부 장관으로 일하다가 내년 총선에 출마하기 위해 지난달 12일 당으로 복귀했다. 박 대통령이 당으로 돌아간 지 한 달여 만에 유 후보자를 3기 경제팀 수장에 내정하고 다시 내각으로 불러들인 것은 국회와 소통을 강화하겠다는 의지가 담겨 있다는 게 청와대 측 설명이다.박 대통령, 경제 챙기겠다는 의지박 대통령은 부총리 후임에 임종룡 금융위원장을 비롯한 관료들과 유 후보자 등 정치인 출신을 놓고 막판까지 고심한 것으로 알려졌다. 박 대통령이 유 후보자를 낙점한 것은 핵심 경제 정책이 번번이 국회에 발목이 잡힌 상황을 고려했다는 게 청와대 안팎의 분석이다.박 대통령은 19대 마지막 정기국회에서 서비스산업발전기본법, 기업활력제고법 등 핵심 민생법안의 처리가 무산되면서 대(對)국회 소통 강화의 필요성을 절감한 것으로 전해졌다. 청와대 관계자는 “유 후보자는 박 대통령의 국정철학을 누구보다 잘 이해하고 공유하고 있을 뿐 아니라 여야 모두와 원만한 관계를 맺고 있다”고 말했다. 또 “친박계 핵심인 유 후보자를 발탁한 것은 박 대통령이 경제정책만큼은 직접 챙기겠다는 의지가 담겨 있다”고 했다. 박 대통령의 경제정책 ‘심복’으로 알려진 안종범 청와대 경제수석을 일선 부처 장관으로 발탁하지 않고 유임시킨 것도 같은 맥락이다.장관 5명 교체 … 총선용 개각이번 개각의 또 다른 특징은 관료와 전문가를 중용했다는 것이다. 집권 4년차를 안정적으로 끌고 가기 위해 정책의 연속성과 전문성을 고려했다고 볼 수 있다. 주형환 산업통상자원부 장관 후보자는 기재부 출신 정통 관료이며, 홍윤식 행정자치부 장관 후보자는 총리실에서 국정운영1실장, 국무1차장(차관급) 등을 지낸 정책기획통이다. 이준식 부총리 겸 교육부 장관 후보자는 약 20년간

In [ ]:
print(df_with_citation['docs'][0][4])

박근혜 대통령 당선인은 첫 장관들을 관료 교수 연구원 출신 전문가로 선택했다. 내각 후보자 18명 가운데 사법·행정·외무·기술고시 등 국가시험을 통과한 인사들이 11명에 달했고, 나머지도 대부분 교수나 연구원 출신이었다. 미국에서 석·박사를 받은 후보자도 9명에 달했다. ‘검증된 전문가를 쓴다’는 박 당선인의 인사 원칙도 재확인됐다.17일 확정된 박근혜 정부의 첫 내각을 분석한 결과 현오석 경제부총리(14회)를 비롯해 서남수 교육부(22회)·유정복 안전행정부(23회)·유진룡 문화체육관광부(22회)·윤상직 산업통상자원부(25회) 장관 후보자는 행정고시 출신이다. 정홍원 총리(14회), 황교안 법무부(23회)·진영 보건복지부(17회)·조윤선 여성가족부(33회) 장관 후보자는 사법고시, 윤병세 외교부·윤성규 환경부 장관 후보자는 각각 외무고시(10회)와 기술고시(13회) 출신이다. 국가시험 출신이 아니라도 육군사관학교(김병관 국방부 장관 후보자)를 나왔거나 김종훈 미래창조과학부·류길재 통일부·이동필 농림축산부·방하남 고용노동부·서승환 국토교통부·윤진숙 해양수산부 장관 후보자 등은 대학 교수나 연구기관의 연구원 출신이다.미국에서 석·박사 학위를 받은 후보자들도 10명이었다. 현오석(펜실베이니아대) 김종훈(메릴랜드대) 서남수(일리노이대) 윤병세(존스홉킨스대) 이동필(미주리대) 윤상직·방하남(위스콘신대) 진영(워싱턴주립대) 조윤선(컬럼비아대) 서승환(프린스턴대) 등이 미국 대학 석·박사를 받았다.MB정부 첫 내각에서 3명이었던 고려대 출신은 류길재 후보자 한 명으로 줄었다. 서울대 출신이 7명이었다. 박 당선인의 모교인 서강대 출신은 없었고, 윤진숙 후보자가 나온 부산여대는 첫 장관 후보자를 배출했다.서울 소재 명문고 비율은 더 높아졌다. MB정부 첫 내각에서 각각 3명과 2명이었던 경기고·서울고 출신은 9명으로 늘었다. 현오석·윤병세·황교안·김병관·진영 후보자는 경기고 출신이고, 서승환·유진룡·방하남·서남수 후보자는 서울고를 졸업했다. 이 중 서승환·유진룡·방하남 후보

In [ ]:
print(df_with_citation['answer'][0])

아베 신조 총리의 개각과 박근혜 대통령의 개각은 각각 다른 정치적 배경과 목적을 가지고 이루어졌습니다.

먼저 아베 신조 총리의 개각은 19명의 각료 중 10명을 교체하는 내용으로, 핵심 각료 9명은 유임시켰습니다. 아베 총리는 자유민주당 총재 연임을 확정한 뒤 ‘1억 총활약사회’라는 구호를 내놓았는데, 이는 일본인 전체가 각 분야에서 활약할 수 있는 사회를 만들고 50년 뒤에도 인구 1억명을 유지한다는 목표를 담고 있습니다. 이번 개각에서는 이 구호를 실현하기 위한 인사들이 발탁되었으며, 극우 성향의 인물도 포함되어 있습니다. 즉, 아베 총리의 개각은 자신의 정책 구호인 ‘1억 총활약사회’ 실현과 연임 후 정책 추진 동력을 확보하는 데 목적이 있었습니다[[doc1]], [[doc3]].

반면 박근혜 대통령의 개각은 정치인 출신 장관을 대거 기용하는 특징이 있습니다. 집권 3년차에 접어들면서 국정 성과를 내야 한다는 판단 아래, 추진력이 강한 정치인 장관을 중용하여 경제혁신과 구조개혁 과제에 집중하고자 했습니다. 또한, 청와대와 여당 내에서 인적 쇄신 요구가 있었으나 이를 무마하고 당·청 소통을 강화하려는 의도도 있었습니다. 특히, 국회에서 법안 통과가 관건인 상황에서 정치인 장관이 유리하다는 점도 고려되었습니다[[doc2]].

또한 박 대통령의 중폭 개각에서는 친박계 핵심 인사를 부총리 겸 기획재정부 장관에 발탁하는 등 경제 정책을 직접 챙기겠다는 의지가 반영되었고, 총선 출마 예정인 현직 장관들을 교체하는 ‘총선용 개각’의 성격도 강했습니다. 이와 함께 관료와 전문가를 중용하여 정책의 연속성과 전문성을 유지하려는 목적도 있었습니다[[doc4]].

요약하면, 아베 신조 총리의 개각은 자신의 정책 구호 실현과 연임 후 정책 추진 동력 확보에 중점을 둔 반면, 박근혜 대통령의 개각은 국정 성과 창출과 국회와의 소통 강화, 총선 대비 및 친박계 강화 등 정치적 목적이 복합적으로 작용한 개각이었다고 할 수 있습니다[[doc1]], [[doc2]], [[doc4